In [1]:
import os, sys, time
from datetime import datetime
import pandas as pd
import numpy as np
import geojson, json
import xarray as xr
import rioxarray

from sklearn.cross_decomposition import CCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
#from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, RepeatedKFold, LeaveOneOut, LeavePOut, KFold, cross_val_predict
from sklearn.metrics import r2_score, mean_squared_error, roc_auc_score, mean_absolute_percentage_error, mean_squared_error, explained_variance_score
from sklearn.base import BaseEstimator, RegressorMixin
from rasterstats import zonal_stats
from matplotlib import pyplot as plt
import matplotlib.colors as colors
import cartopy.crs as ccrs
import importlib
from PyQt5 import QtWidgets, uic, QtCore
from PyQt5.QtWidgets import QFileDialog
import traceback

from pathlib import Path

import gl
from dev_functions.dev_functions import *

# derived
gl.maxLeadTime=6
gl.configFile="forecast_config.json"


def readGUI():
    #defined parameters/variables
    gl.config['rootDir']=gl.window.lineEditDirectory.text()
    gl.config['fcstTargetYear']=int(gl.window.lineEdit_tgtyear.text())
    gl.config['predictorYear']=int(gl.window.lineEdit_srcyear.text())
    gl.config['predictorMonth']=gl.window.comboBox_srcmon.currentText()
    gl.config['fcstTargetSeas']=gl.window.comboBox_tgtseas.currentText()
    gl.config['climStartYr']=int(gl.window.lineEdit_climstartyr.text())
    gl.config['climEndYr']=int(gl.window.lineEdit_climendyr.text())
    gl.config['timeAggregation']=gl.window.comboBox_timeaggregation.currentText()

    gl.config["predictandFileName"]=gl.window.lineEdit_predictandfile.text()
    gl.config["predictandVar"]=gl.window.comboBox_predictandvar.currentText()

    gl.config["zonesFile"]=gl.window.lineEdit_zonesfile.text()
    gl.config["zonesAttribute"]=gl.window.comboBox_zonesattribute.currentText()
    gl.config["zonesAggregate"]=gl.window.checkBox_zonesaggregate.isChecked()

    gl.config["overlayFile"]=gl.window.lineEdit_overlayfile.text()

    
    gl.config["predictorExtents"]=[]
    for model in range(5):
        temp={}
        for var in ["minLon","maxLon","minLat","maxLat"]:
            itemName="lineEdit_{}{}".format(var.lower(), model)
            if hasattr(gl.window, itemName):
                item=getattr(gl.window, itemName, None)
                temp[var]=item.text()
        if len(temp)==4:
            gl.config["predictorExtents"].append(temp)
    
    gl.config["predictorFiles"]=[]
    for model in range(5):
        temp=[]
        itemName="lineEdit_predictorfile{}".format(model)
        if hasattr(gl.window, itemName):
            temp.append(getattr(gl.window, itemName, None).text())
        itemName="comboBox_predictorvar{}".format(model)
        if hasattr(gl.window, itemName):
            temp.append(getattr(gl.window, itemName, None).currentText())
        if len(temp)==2:
            gl.config["predictorFiles"].append(temp)

    gl.config["crossval"]=[]
    for model in range(5):
        itemName="comboBox_crossval{}".format(model)
        if hasattr(gl.window, itemName):
            gl.config["crossval"].append(getattr(gl.window, itemName, None).currentData())
                
    gl.config["preproc"]=[]
    for model in range(5):
        itemName="comboBox_preproc{}".format(model)
        if hasattr(gl.window, itemName):
            gl.config["preproc"].append(getattr(gl.window, itemName, None).currentData())
                
    gl.config["regression"]=[]
    for model in range(5):
        itemName="comboBox_regression{}".format(model)
        if hasattr(gl.window, itemName):
            gl.config["regression"].append(getattr(gl.window, itemName, None).currentData())

            
    #derived variables
    
    #set target type 
    if gl.config["zonesAggregate"]:
        gl.targetType="zones"
    elif gl.config["predictandFileName"][-3:]=="csv":
        gl.targetType="points"
    else:
        gl.targetType="grid"

    if len(gl.config["fcstTargetSeas"])>3:
        gl.fcstBaseTime="seas"
    else:
        gl.fcstBaseTime="mon"

        
def saveConfig():
    #defined parameters/variables
    with open(gl.configFile, "w") as f:
        json.dump(gl.config, f, indent=4)
        showMessage("saved config to: {}".format(gl.configFile), "INFO")
    
def computeModel(model):
    
    readGUI()    
    saveConfig()
    
    showMessage(model, "INFO")
    leadTime=getLeadTime()
    
    if leadTime is None:
        showMessage("Lead time could not be calculated, stopping early.", "ERROR")
        return
    
    #reading predictors data
    predictor=readPredictor(model)
    if predictor is None:
        showMessage("Predictor could not be read, stopping early.", "ERROR")
        return

    #reading predictand data - this will calculate seasonal from monthly if needed.
    predictand, geoData=readPredictand()
    if predictand is None:
        showMessage("Predictand could not be read, stopping early.", "ERROR")
        return

    if gl.config["zonesAggregate"]:
        zonesVector=gpd.read_file(gl.config["zonesFile"])
        showMessage("Aggregating data to zones read from {} ...".format(gl.config["zonesFile"]))
        predictand,geoData=aggregatePredictand(predictand, geoData, zonesVector)
    else:
        zonesVector=None
        
    #defining target date for forecast. If seasonal - then this is the first month of the season.
    fcstTgtDate=pd.to_datetime("01 {} {}".format(gl.config['fcstTargetSeas'][0:3], gl.config['fcstTargetYear']))
    
    #do not need this anymore
    #gl.config["fcstTgtCode"]=seasons[fcstTgtDate.month-1]
    
    #finding overlap of predictand and predictor
    showMessage("Aligning predictor and predictand data...")
    predictandHcst,predictorHcst=getHcstData(predictand,predictor)
    predictorFcst=getFcstData(predictor)
    if predictandHcst is None:
        showMessage("Hindcast data for predictand could not be derived, stopping early.", "ERROR")
        return

    showMessage("Setting up directories to write to...")        
    forecastID="{}_{}".format(gl.predictorDate.strftime("%Y%m"), gl.config['fcstTargetSeas'])
    forecastDir=Path(gl.config['rootDir'], forecastID, gl.targetType, "{}_{}_{}".format(gl.config["preproc"][model],gl.config["regression"][model],gl.config["crossval"][model]))

    mapsDir=Path(forecastDir, "maps")
    timeseriesDir=Path(forecastDir, "timeseries")
    outputDir=Path(forecastDir, "output")
    diagsDir=Path(forecastDir, "diagnostics")

    for adir in [mapsDir,outputDir, diagsDir,timeseriesDir]:
        if not os.path.exists(adir):
            showMessage("\toutput directory {} does not exist. creating...".format(adir))
            os.makedirs(adir)
                
    #calculaing observed terciles
    #is there a need to do a strict control of overlap???
    result=getObsTerciles(predictand, predictandHcst)
    if result is None:
        showMessage("Terciles could not be calculated, stopping early.", "ERROR")
                
    obsTercile,tercThresh=result


    #setting up cross-validation
    cvkwargs=crossvalidator_config[gl.config['crossval'][model]][1]
    cv=crossvalidators[gl.config['crossval'][model]](**cvkwargs)
    
    
    #arguments for regressor
    kwargs=regressor_config[gl.config['regression'][model]][1]

    if gl.config['preproc'][model]=="PCR":
        #regession model
        regressor = PCRegressor(regressor_name=gl.config['regression'][model], **kwargs)
    elif gl.config['preproc'][model]=="CCA":
        showMessage("sorry, CCA is not implemented yet", "NONCRITICAL")
        return
#        regressor = CCAregressor(regressor_name=gl.config['regression'][model], **kwargs)
    else:
        showMessage("Preprocessing method not available", "ERROR")
        return
  
    #cross-validated hindcast
    showMessage("Calculating cross-validated hindcast...")
    cvHcst = cross_val_predict(regressor,predictorHcst,  predictandHcst, cv=cv)
    cvHcst=pd.DataFrame(cvHcst, index=predictandHcst.index, columns=predictandHcst.columns)

    
    #actual prediction
    showMessage("Calculating deteriministic forecast...")
    regressor.fit(predictorHcst,  predictandHcst)
    detFcst=regressor.predict(predictorFcst)
    detFcst=pd.DataFrame(detFcst, index=[fcstTgtDate], columns=predictandHcst.columns)
    
    #calculate forecast anomalies
    refData=predictand[str(gl.config["climStartYr"]):str(gl.config["climEndYr"])]   
    detFcst=getFcstAnomalies(detFcst,refData)
    
    #deriving probabilistic prediction
    showMessage("Calculating probabilistic hindcast and forecast using error variance...")
    result=probabilisticForecast(cvHcst, predictandHcst,detFcst["forecast"],tercThresh)
    if result is None:
        showMessage("Probabilistic forecast could not be calculated", "ERROR")
        return
    probFcst,probHcst=result
    showMessage("Hindcast and forecast calculated.")
    
    
    #calculating skill
    showMessage("Calculating skill scores...")
    scores=getSkill(probHcst,cvHcst,predictandHcst,obsTercile)    
    if scores is None:
        showMessage("Skill could not be calculated", "ERROR")
        return
    
    #saving data
    
    
    showMessage("Plotting forecast maps...")    
    #plotting forecast
    if gl.targetType=="grid":
        detfcst=detFcst.stack(level=["lat","lon"],future_stack=True).droplevel(0).T
        probfcst=probFcst.stack(level=["lat","lon"],future_stack=True).droplevel(0).T
    else:
        detfcst=detFcst.stack(future_stack=True).droplevel(0).T
        probfcst=probFcst.stack(future_stack=True).droplevel(0).T
        
    plotMaps(detfcst, geoData, mapsDir, forecastID, zonesVector)
    plotMaps(probfcst, geoData, mapsDir, forecastID, zonesVector)
    
    showMessage("Plotting skill maps...")    
    #plotting skill scores
    plotMaps(scores, geoData, mapsDir, forecastID, zonesVector)

    showMessage("Plotting time series...") 
    plotTimeSeries(cvHcst,predictandHcst, detFcst, tercThresh, timeseriesDir, forecastID)
    
    showMessage("All done!", "SUCCESS")    
    
    
    return


def browse(line_edit, mode='file', parent=None, caption="Select File", file_filter="All Files (*)"):
    if mode == 'file':
        path, _ = QFileDialog.getOpenFileName(parent, caption, "", file_filter)
    elif mode == 'dir':
        path = QFileDialog.getExistingDirectory(parent, caption)
    else:
        raise ValueError("Unsupported browse mode")

    if path:
        line_edit.setText(path)  
            
    
class Worker(QtCore.QThread):
    log = QtCore.pyqtSignal(str)
    finished = QtCore.pyqtSignal(str)

    def __init__(self, task_name, task_function, *args, **kwargs):
        super().__init__()
        self.task_name=task_name
        self.task_function = task_function
        self.args = args
        self.kwargs = kwargs

    def run(self):
        """Run the provided function in a thread and emit logs."""
        try:
            self.log.emit(f"Task '{self.task_name}' started...")
            # Run the task
            self.task_function(*self.args, **self.kwargs)
            self.log.emit(f"Task '{self.task_name}' finished successfully.")
        except Exception as e:
            tb = traceback.format_exc()
            self.log.emit(f"Error occurred in {self.task_name}:\n{tb}")            
        finally:
            self.finished.emit(self.task_name)
            

class MainWindow(QtWidgets.QMainWindow):
    log_signal = QtCore.pyqtSignal(str)
    
  
    def __init__(self):
        super().__init__()
        uic.loadUi("forecast.ui", self)
        
        #initialize garbage collector
        self.workers = []
        
        self.log_signal.connect(self.append_log)

        # Collect buttons
        self.buttons = [self.button_run0]

        # Connect signals
        self.button_run0.clicked.connect(lambda _, idx=0: self.start_task(f"Model {idx}", computeModel, idx))
        
        self.clearLogButton.clicked.connect(self.logWindow.clear)
        
        #directory browser
        self.pushButton_rootdir.clicked.connect(
            lambda: browse(self.lineEdit_rootdir, mode='dir', parent=self)
        )

        self.pushButton_predictandfile.clicked.connect(
            lambda: browse(self.lineEdit_predictandfile, mode='file', parent=self, 
                           file_filter="CSV or NetCDF (*.csv *.nc)")
        )

        self.pushButton_zonesfile.clicked.connect(
            lambda: browse(self.lineEdit_zonesfile, mode='file', parent=self, 
                           file_filter="Vector Files (*.shp *.geojson)")
        )
        self.pushButton_overlayfile.clicked.connect(
            lambda: browse(self.lineEdit_overlayfile, mode='file', parent=self, 
                           file_filter="Vector Files (*.shp *.geojson)")
        )

    # ---------- Thread Handling ----------
    def start_task(self, name, func, *args):
        worker = Worker(name, func, *args)
        worker.log.connect(self.log_signal.emit)
        # finished cleans up workers stack
        worker.finished.connect(self.cleanup_worker)
        self.workers.append(worker)  # keep reference
        worker.start()
        
    def append_log(self, message: str):
        self.logWindow.appendHtml(f"<pre>{message}</pre>")
        self.logWindow.ensureCursorVisible()
            
    def cleanup_worker(self, task_name):
        self.workers = [w for w in self.workers if w.isRunning()]
        self.logWindow.appendHtml(f"<i>Task '{task_name}' cleaned up.</i>")

        
    def set_buttons_enabled(self, enabled: bool):
        for btn in self.buttons:
            btn.setEnabled(enabled)


            
    def browse_directory(self):
        dir_path = QFileDialog.getExistingDirectory(self, "Select Directory", "")
        if dir_path:
            self.lineEditDirectory.setText(dir_path)
            
    
if __name__ == "__main__":
    
    #shows the main window
    app = QtWidgets.QApplication(sys.argv)
    gl.window = MainWindow()
    gl.window.show()

    
    

    
    

tgtSeass=["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec","Jan-Mar","Feb-Apr","Mar-May","Apr-Jun","May-Jul","Jun-Aug","Jul-Sep","Aug-Oct","Sep-Nov","Oct-Dec","Nov-Jan","Dec-Feb"]

srcMons=["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]

timeAggregations={"sum","mean"}

crossvalidators = {
        "KF": KFold,
        'LOO': LeaveOneOut,
}

#can be read from json - potentially editable by user
regressors = {
    "OLS":["Linear regression", {}],
    "Lasso":["Lasso regression", {'alpha': 0.01}],
    "Ridge":["Ridge regression", {'alpha': 1.0}],
    "RF":["Random Forest", {'n_estimators': 100, 'max_depth': 5}],
    "MLP":["Multi Layer Perceptron", {'hidden_layer_sizes': (50, 25), 'max_iter': 1000, 'random_state': 0}],
    "Trees":["Decision Trees", {'max_depth': 2}]
}

preprocessors={
    "PCR":["Principal Component Regression (PCR)", {}],
    "CCA":["Canonical Corelation Analysis (CCA)", {}]
}



if os.path.exists(gl.configFile):
    try:
        showMessage("reading config from: {}".format(gl.configFile))
        with open(gl.configFile, "r") as f:
            gl.config = json.load(f)
        populateGui()
    except:    
        showMessage("config file corrupted. Making default config.".format(gl.configFile))
        makeConfig()
        populateGui()
else:
    showMessage("config file {} does not exist. Making default config.".format(gl.configFile))
    makeConfig()
    populateGui()


sys.exit(app.exec_())

SystemExit: 0

/home/piotr/mambaforge/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
